### Atividade (1.0 pt): Visualização de casos de COVID-19 no Piauí (equipe 5 integrantes)

<dl>
<dt>Apresentar um jupyter notebook atendendo os seguintes itens: </dt>
    <dd>1. Filtrar os dados para selecionar apenas aqueles referentes ao estado do Piauí. Se uma cidade possui mais de 1 amostra, selecionar aquela mais recente (nota: as datas estão em ordem decrescente);</dd>
    <dd>2. Plotar um mapa com a localização e número de casos;</dd>
    <dd>3. Indicar o número de amostras, atributos e dados faltantes (se existir) do conjunto de dados;</dd>
    <dd>4. Apresentar as estatísticas descritivas dos atributos não categóricos;</dd>
    <dd>5. Apresentar os histogramas dos atributos não categóricos;</dd>
    <dd>6. Capturar as latitudes e longitudes para cada 1 dos municípios presentes na base de dados e adicionar as colunas latitude e longitude nos dados;</dd>
    <dd>7. Apresentar um gráfico da latitude x longitude, onde cada amostra será representada por um círculo e quanto maior o valor do campo estimated_population, maior o raio do círculo que representa aquela amostra. Já a cor representa a taxa de mortalidade (death_rate), quão maior esse índice, mais próximo do vermelho;</dd>
    <dd>8. Calcular a matriz de correlação;</dd>
    <dd>9. Verificar a correlação entre o atributo death_rate e os demais atributos não categóricos;</dd>
</dl>

## Importando as bibliotecas

## Carregando os dados

## Ponto #1

## Ponto #2

## Ponto #3

## Ponto #4

## Ponto #5

## Ponto #6

## Ponto #7

## Ponto #8

## Ponto #9